In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import os, random, numpy as np, pandas as pd, timm
import matplotlib.pyplot as plt

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
class InferenceDataset(Dataset):
    def __init__(self, df, base_path, transform=None):
        self.df = df
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.base_path, row["image_path"])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
# Model
class MultiRegressionModel(nn.Module):
    def __init__(self, model_name="efficientnet_b2", output_dim=5):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False, num_classes=output_dim)

    def forward(self, x):
        return self.model(x)


In [ ]:
def tta_inference(model, images):
    preds = model(images)
    preds_lr = model(torch.flip(images, dims=[3]))
    preds_ud = model(torch.flip(images, dims=[2]))
    preds_lrud = model(torch.flip(images, dims=[2, 3]))
    return (preds + preds_lr + preds_ud + preds_lrud) / 4.0

In [ ]:
def run_inference(model_paths, test_csv, base_path="/kaggle/input/csiro-biomass", batch_size=8):
    test_df = pd.read_csv(test_csv)
    test_df = test_df[["sample_id", "image_path"]].drop_duplicates().reset_index(drop=True)
    test_df["sample_id"] = test_df["sample_id"].astype(str)

    transform = T.Compose([
        T.Resize((512, 512)),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    dataset = InferenceDataset(test_df, base_path, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    results = []

    for fold, model_path in enumerate(model_paths):
        model = MultiRegressionModel("efficientnet_b2", 5)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device).eval()

        preds_all = []
        with torch.no_grad():
            for imgs in loader:
                imgs = imgs.to(device)
                preds = tta_inference(model, imgs)
                preds_all.append(preds.cpu().numpy())
        results.append(np.concatenate(preds_all))

    preds_mean = np.mean(results, axis=0)
    out_df = pd.DataFrame(preds_mean, columns=["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"])
    out_df["sample_id"] = test_df["sample_id"]

    sub = out_df.melt(id_vars="sample_id", var_name="target_name", value_name="target")
    sub["sample_id"] = sub["sample_id"].astype(str) + "__" + sub["target_name"]
    sub["target"] = sub["target"].clip(0, 200)
    sub = sub[["sample_id", "target"]]
    sub.to_csv("submission.csv", index=False)
    print("Saved submission.csv")
    return sub


In [ ]:
def visualize_samples(df, base_path, n=3):
    import os
    from PIL import Image
    import matplotlib.pyplot as plt

    fig, axes = plt.subplots(1, n, figsize=(15, 5))
    for i, ax in enumerate(axes):
        img_path = df.iloc[i]["image_path"]
        if img_path.startswith("test/"):
            path = os.path.join("/kaggle/input/csiro-biomass", img_path)
        else:
            path = os.path.join(base_path, img_path)
        img = Image.open(path)
        ax.imshow(img)
        ax.axis("off")
    plt.show()



In [ ]:
if __name__ == "__main__":
    model_paths = [
        "/kaggle/input/biomass-prediction/transformers/default/1/model_fold0.pth",
        "/kaggle/input/biomass-prediction/transformers/default/1/model_fold1.pth",
        "/kaggle/input/biomass-prediction/transformers/default/1/model_fold2.pth"
    ]
    sub = run_inference(model_paths, "/kaggle/input/csiro-biomass/test.csv")
    visualize_samples(pd.read_csv("/kaggle/input/csiro-biomass/test.csv").head(3),
                      "/kaggle/input/csiro-biomass/test")